In [1]:
import ast
import json
import os
import shutil
import albumentations
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom as dicom
import tensorflow as tf
import torch
from sklearn.model_selection import StratifiedKFold
from PIL import Image
from tensorflow import keras
from tqdm import tqdm
from src.utils import Aug, Generator, GetModel
import tensorflow_addons as tfa
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
labels = pd.read_csv('/app/_data/additional_dataset/stage_2_train_labels.csv')
class_info = pd.read_csv('/app/_data/additional_dataset/stage_2_detailed_class_info.csv')
IMG_PATH = '/app/_data/additional_dataset/stage_2_train_images/'

# data

In [3]:
labels

,patientId,x,y,width,height,Target
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1
...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1


In [4]:
class_info

,patientId,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,No Lung Opacity / Not Normal
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,No Lung Opacity / Not Normal
2,00322d4d-1c29-4943-afc9-b6754be640eb,No Lung Opacity / Not Normal
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,Normal
4,00436515-870c-4b36-a041-de91049b9ab4,Lung Opacity
...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,Lung Opacity
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,Normal
30224,c1f6b555-2eb1-4231-98f6-50a963976431,Normal
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,Lung Opacity


In "labels" "Target" column show if there is Lung Opacity and bounding box in image or not

# collecting metadata about images

In [5]:
# for ix in tqdm(labels.index.tolist()):
#     img_name =  labels.loc[ix, "patientId"]
#     img_path = os.path.join(IMG_PATH, img_name+'.dcm')
#     data_file = dicom.dcmread(img_path)
#     labels.loc[ix, "modality"] = data_file.Modality
#     labels.loc[ix, "PatientSex"] = data_file.PatientSex
#     labels.loc[ix, "BodyPartExamined"] = data_file.BodyPartExamined
#     labels.loc[
#         ix, "PhotometricInterpretation"
#     ] = data_file.PhotometricInterpretation
#     labels.loc[ix, "img_height"] = data_file.Rows
#     labels.loc[ix, "img_width"] = data_file.Columns
# labels.to_csv('/app/_data/additional_dataset/labels_full.csv', index=False)

100%|██████████| 30227/30227 [02:25<00:00, 207.18it/s]


In [17]:
labels = pd.read_csv('/app/_data/additional_dataset/labels_full.csv')

In [7]:
labels['modality'].unique()
labels['img_width'].unique()
labels['img_height'].unique()
labels['PhotometricInterpretation'].unique()
labels['PatientSex'].unique()
labels['BodyPartExamined'].unique()

array(['CR'], dtype=object)

array([1024.])

array([1024.])

array(['MONOCHROME2'], dtype=object)

array(['F', 'M'], dtype=object)

array(['CHEST'], dtype=object)

In [18]:
labels

,patientId,x,y,width,height,Target,modality,PatientSex,BodyPartExamined,PhotometricInterpretation,img_height,img_width,x_center,y_center,w,h,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,0.361816,0.333496,0.208008,0.370117,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,0.291992,0.476074,0.222656,0.370117,0
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,0.684082,0.552246,0.254883,0.336914,0


In [9]:
labels['x_center'] = (labels['x']+labels['width']/2)/labels['img_width']
labels['y_center'] = (labels['y']+labels['height']/2)/labels['img_height']
labels['w'] = labels['width']/labels['img_width']
labels['h'] = labels['height']/labels['img_height']
labels['class'] = 0

In [10]:
labels.to_csv('/app/_data/additional_dataset/labels_full.csv', index=False)

In [11]:
train = labels.copy()
for ix in tqdm(train.index.tolist()):
    img_name =  labels.loc[ix, "patientId"]
    img_path = os.path.join(IMG_PATH, img_name+'.dcm')
    img_type = train.loc[ix, 'PhotometricInterpretation']
    data = dicom.dcmread(img_path)
    img = dicom.pixel_data_handlers.util.apply_voi_lut(data.pixel_array, data)
    if img_type == "MONOCHROME1":
        img = img.max() - img
    img = (img - img.min()) / (img.max() - img.min())
    img = (np.array(img) * 255).astype("uint8")
    img = np.stack([img, img, img], axis=-1)
    img = Image.fromarray(img)
    img.save("/app/_data/jpg/"+img_name+'.jpg')

100%|██████████| 30227/30227 [21:09<00:00, 23.80it/s]


In [15]:
shutil.rmtree('/app/_data/models/EffB7_2')

In [16]:
labels

,patientId,x,y,width,height,Target,modality,PatientSex,BodyPartExamined,PhotometricInterpretation,img_height,img_width,x_center,y_center,w,h,class
0,0004cfab-14fd-4e49-80ba-63a80b6bddd6,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
1,00313ee0-9eaa-42f4-b0ab-c148ed3241cd,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
2,00322d4d-1c29-4943-afc9-b6754be640eb,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
3,003d8fa0-6bf1-40ed-b54c-ac657f8495c5,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
4,00436515-870c-4b36-a041-de91049b9ab4,264.0,152.0,213.0,379.0,1,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,0.361816,0.333496,0.208008,0.370117,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30222,c1ec14ff-f6d7-4b38-b0cb-fe07041cbdc8,185.0,298.0,228.0,379.0,1,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,0.291992,0.476074,0.222656,0.370117,0
30223,c1edf42b-5958-47ff-a1e7-4f23d99583ba,NaN,NaN,NaN,NaN,0,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
30224,c1f6b555-2eb1-4231-98f6-50a963976431,NaN,NaN,NaN,NaN,0,CR,M,CHEST,MONOCHROME2,1024.0,1024.0,NaN,NaN,NaN,NaN,0
30225,c1f7889a-9ea9-4acb-b64c-b737c929599a,570.0,393.0,261.0,345.0,1,CR,F,CHEST,MONOCHROME2,1024.0,1024.0,0.684082,0.552246,0.254883,0.336914,0
